In [1]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
db_path = data_root / "건축물대장_2024.db"

In [2]:
columns_기본개요 = {
    "관리_건축물대장_PK": "VARCHAR",
    "관리_상위_건축물대장_PK": "VARCHAR",
    "대장_구분_코드": "VARCHAR",
    "대장_구분_코드_명": "VARCHAR",
    "대장_종류_코드": "VARCHAR",
    "대장_종류_코드_명": "VARCHAR",
    "대지_위치": "VARCHAR",
    "도로명_대지_위치": "VARCHAR",
    "건물_명": "VARCHAR",
    "시군구_코드": "VARCHAR",
    "법정동_코드": "VARCHAR",
    "대지_구분_코드": "CHAR",  # an alias to VARCHAR
    "번": "VARCHAR",
    "지": "VARCHAR",
    "특수지_명": "VARCHAR",
    "블록": "VARCHAR",
    "로트": "VARCHAR",
    "외필지_수": "INTEGER",
    "새주소_도로_코드": "VARCHAR",
    "새주소_법정동_코드": "VARCHAR",
    "새주소_지상지하_코드": "VARCHAR",
    "새주소_본_번": "INTEGER",
    "새주소_부_번": "INTEGER",
    "지역_코드": "VARCHAR",
    "지구_코드": "VARCHAR",
    "구역_코드": "VARCHAR",
    "지역_코드_명": "VARCHAR",
    "지구_코드_명": "VARCHAR",
    "구역_코드_명": "VARCHAR",
    "생성_일자": "VARCHAR",
}

In [3]:
columns_표제부 = {
    "관리_건축물대장_PK": "VARCHAR",
    "대장_구분_코드": "VARCHAR",
    "대장_구분_코드_명": "VARCHAR",
    "대장_종류_코드": "VARCHAR",
    "대장_종류_코드_명": "VARCHAR",
    "대지_위치": "VARCHAR",
    "도로명_대지_위치": "VARCHAR",
    "건물_명": "VARCHAR",
    "시군구_코드": "VARCHAR",
    "법정동_코드": "VARCHAR",
    "대지_구분_코드": "VARCHAR",
    "번": "VARCHAR",
    "지": "VARCHAR",
    "특수지_명": "VARCHAR",
    "블록": "VARCHAR",
    "로트": "VARCHAR",
    "외필지_수": "INTEGER",
    "새주소_도로_코드": "VARCHAR",
    "새주소_법정동_코드": "VARCHAR",
    "새주소_지상지하_코드": "VARCHAR",
    "새주소_본_번": "INTEGER",
    "새주소_부_번": "INTEGER",
    "동_명": "VARCHAR",
    "주_부속_구분_코드": "CHAR",  # an alias to VARCHAR
    "주_부속_구분_코드_명": "VARCHAR",
    "대지_면적(㎡)": "DECIMAL",  # replacement for NUMERIC(19,9),
    # The default is DECIMAL(18, 3), if none are specified.
    "건축_면적(㎡)": "DECIMAL",
    "건폐_율(%)": "DECIMAL",
    "연면적(㎡)": "DECIMAL",
    "용적_률_산정_연면적(㎡)": "DECIMAL",
    "용적_률(%)": "DECIMAL",
    "구조_코드": "VARCHAR",
    "구조_코드_명": "VARCHAR",
    "기타_구조": "VARCHAR",
    "주_용도_코드": "VARCHAR",
    "주_용도_코드_명": "VARCHAR",
    "기타_용도": "VARCHAR",
    "지붕_코드": "VARCHAR",
    "지붕_코드_명": "VARCHAR",
    "기타_지붕": "VARCHAR",
    "세대_수(세대)": "INTEGER",
    "가구_수(가구)": "INTEGER",
    "높이(m)": "DECIMAL",
    "지상_층_수": "INTEGER",
    "지하_층_수": "INTEGER",
    "승용_승강기_수": "INTEGER",
    "비상용_승강기_수": "INTEGER",
    "부속_건축물_수": "INTEGER",
    "부속_건축물_면적(㎡)": "DECIMAL",
    "총_동_연면적(㎡)": "DECIMAL",
    "옥내_기계식_대수(대)": "INTEGER",
    "옥내_기계식_면적(㎡)": "DECIMAL",
    "옥외_기계식_대수(대)": "INTEGER",
    "옥외_기계식_면적(㎡)": "DECIMAL",
    "옥내_자주식_대수(대)": "INTEGER",
    "옥내_자주식_면적(㎡)": "DECIMAL",
    "옥외_자주식_대수(대)": "INTEGER",
    "옥외_자주식_면적(㎡)": "DECIMAL",
    "허가_일": "VARCHAR",
    "착공_일": "VARCHAR",
    "사용승인_일": "VARCHAR",
    "허가번호_년": "VARCHAR",
    "허가번호_기관_코드": "CHAR",
    "허가번호_기관_코드_명": "VARCHAR",
    "허가번호_구분_코드": "VARCHAR",
    "허가번호_구분_코드_명": "VARCHAR",
    "호_수(호)": "INTEGER",
    "에너지효율_등급": "VARCHAR",
    "에너지절감_율": "DOUBLE",  # replacement for NUMERIC(19,9),
    # when it is not expected to be summed
    "에너지_EPI점수": "INTEGER",
    "친환경_건축물_등급": "CHAR",
    "친환경_건축물_인증점수": "INTEGER",
    "지능형_건축물_등급": "CHAR",
    "지능형_건축물_인증점수": "INTEGER",
    "생성_일자": "VARCHAR",
    "내진_설계_적용_여부": "VARCHAR",
    "내진_능력": "VARCHAR",
}

In [4]:
columns_층별개요 = {
    "관리_건축물대장_PK": "VARCHAR",
    "대지_위치": "VARCHAR",
    "도로명_대지_위치": "VARCHAR",
    "건물_명": "VARCHAR",
    "시군구_코드": "VARCHAR",
    "법정동_코드": "VARCHAR",
    "대지_구분_코드": "VARCHAR",
    "번": "VARCHAR",
    "지": "VARCHAR",
    "특수지_명": "VARCHAR",
    "블록": "VARCHAR",
    "로트": "VARCHAR",
    "새주소_도로_코드": "VARCHAR",
    "새주소_법정동_코드": "VARCHAR",
    "새주소_지상지하_코드": "VARCHAR",
    "새주소_본_번": "INTEGER",
    "새주소_부_번": "INTEGER",
    "동_명": "VARCHAR",
    "층_구분_코드": "VARCHAR",
    "층_구분_코드_명": "VARCHAR",
    "층_번호": "INTEGER",
    "층_번호_명": "VARCHAR",
    "구조_코드": "VARCHAR",
    "구조_코드_명": "VARCHAR",
    "기타_구조": "VARCHAR",
    "주_용도_코드": "VARCHAR",
    "주_용도_코드_명": "VARCHAR",
    "기타_용도": "VARCHAR",
    "면적(㎡)": "DECIMAL",
    "주_부속_구분_코드": "VARCHAR",
    "주_부속_구분_코드_명": "VARCHAR",
    "면적_제외_여부": "VARCHAR",
    "생성_일자": "VARCHAR",
}

In [5]:
# Open a DuckDB connection
con = duckdb.connect(database=db_path)

# print the list of tables in the database
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])
    print()

# show heads of the tables
for table in tables:
    table_name = table[0]
    print(f"Head of {table_name}:")
    df = con.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchdf()
    print(df)
    print()

Tables in the database:
기본개요

층별개요

표제부

Head of 기본개요:
  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0      100211           None        1         일반        1      총괄표제부   
1      100215           None        1         일반        1      총괄표제부   
2      100216           None        1         일반        1      총괄표제부   
3      100218           None        1         일반        1      총괄표제부   
4      100219           None        1         일반        1      총괄표제부   

                  대지_위치                  도로명_대지_위치  건물_명 시군구_코드  ...  \
0  서울특별시 종로구 관철동 12-1번지   서울특별시 종로구 종로14길 20 (관철동)  None  11110  ...   
1  서울특별시 종로구 청진동 21-1번지  서울특별시 종로구 종로5길 32-6 (청진동)  None  11110  ...   
2  서울특별시 종로구 중학동 16-2번지                       None  None  11110  ...   
3  서울특별시 종로구 중학동 24-1번지      서울특별시 종로구 율곡로 5 (중학동)  None  11110  ...   
4  서울특별시 종로구 중학동 35-1번지                       None  None  11110  ...   

  새주소_지상지하_코드 새주소_본_번 새주소_부_번 지역_코드 지구_코드 구역_코드 지역_코드_명  지구_코드_명 구역_코드_명  \
0  